In [1]:
file = open("text.txt", "r")
lines = file.read()

In [10]:
lines

'WNT signalling orchestrates a number of developmental programs. In response to this stimulus, cytoplasmic beta-catenin (encoded by CTNNB1) is stabilized, enabling downstream transcriptional activation by members of the LEF/TCF family. One of the target genes for beta-catenin/TCF encodes c-MYC, explaining why constitutive activation of the WNT pathway can lead to cancer, particularly in the colon. Most colon cancers arise from mutations in the gene encoding adenomatous polyposis coli (APC), a protein required for ubiquitin-mediated degradation of beta-catenin, but a small percentage of colon and some other cancers harbour beta-catenin-stabilizing mutations. Recently, we discovered that transgenic mice expressing an activated beta-catenin are predisposed to developing skin tumours resembling pilomatricomas. Given that the skin of these adult mice also exhibits signs of de novo hair-follicle morphogenesis, we wondered whether human pilomatricomas might originate from hair matrix cells an

In [2]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import text_to_word_sequence

Using Theano backend.


In [12]:
# estimate the size of the vocabulary
words = set(text_to_word_sequence(lines))
vocab_size = len(words)
print(vocab_size)
words


8265


{'tfr',
 'died',
 '001',
 'founding',
 'occur',
 'comprising',
 'devised',
 'dxs148',
 '134',
 'arg170',
 'catalyzes',
 'bacteriolytic',
 'monoallelic',
 'adrenocorticotropic',
 'maori',
 'dysmyelinating',
 'clcn1',
 '551c',
 'phagocyte',
 'samine',
 'world',
 'effects',
 'torsion',
 'blida',
 'dilated',
 'psoriasis',
 'translucency',
 'g5260a',
 'huntingtin',
 'medulloblastoma',
 'sottas',
 'dominantly',
 'markers',
 'prognostic',
 'preparations',
 'cochin',
 'ivs5',
 'selection',
 'encompasses',
 'represented',
 'incomplete',
 'leukoencephalopathy',
 'investigations',
 'threefold',
 'r330s',
 'quite',
 'globe',
 'considerable',
 'nlaiii',
 'subtotal',
 'revertants',
 'fibroepithelial',
 'd4s10',
 'caucasian',
 'lack',
 'peripheral',
 'progestagens',
 'children',
 'functioned',
 'c20',
 'nonconsanguineous',
 'immediately',
 'tetracycline',
 'polynesians',
 'crna',
 'q33',
 'gly',
 'dxys1',
 'quantified',
 'processing',
 'complementation',
 'accounts',
 'undergone',
 'identity',
 'chil

In [11]:
words

NameError: name 'words' is not defined

In [13]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', 
                       lower=True, split=' ', char_level=False, oov_token=None)

tokens = tokenizer.fit_on_texts(words)
tokens

In [15]:
print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_index)
print(tokenizer.word_docs)

OrderedDict([('tfr', 1), ('died', 1), ('001', 1), ('founding', 1), ('occur', 1), ('comprising', 1), ('devised', 1), ('dxs148', 1), ('134', 1), ('arg170', 1), ('catalyzes', 1), ('bacteriolytic', 1), ('monoallelic', 1), ('adrenocorticotropic', 1), ('maori', 1), ('dysmyelinating', 1), ('clcn1', 1), ('551c', 1), ('phagocyte', 1), ('samine', 1), ('world', 1), ('effects', 1), ('torsion', 1), ('blida', 1), ('dilated', 1), ('psoriasis', 1), ('translucency', 1), ('g5260a', 1), ('huntingtin', 1), ('medulloblastoma', 1), ('sottas', 1), ('dominantly', 1), ('markers', 1), ('prognostic', 1), ('preparations', 1), ('cochin', 1), ('ivs5', 1), ('selection', 1), ('encompasses', 1), ('represented', 1), ('incomplete', 1), ('leukoencephalopathy', 1), ('investigations', 1), ('threefold', 1), ('r330s', 1), ('quite', 1), ('globe', 1), ('considerable', 1), ('nlaiii', 1), ('subtotal', 1), ('revertants', 1), ('fibroepithelial', 1), ('d4s10', 1), ('caucasian', 1), ('lack', 1), ('peripheral', 1), ('progestagens', 1

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


In [41]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(lines)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(lines)
print(encoded_docs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
from keras.preprocessing.text import one_hot
encoded_docs = [one_hot(d, vocab_size) for d in lines]

In [42]:
# pad documents 
max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[20  0  0 ...  0  0  0]
 [ 5  0  0 ...  0  0  0]
 [ 2  0  0 ...  0  0  0]
 ...
 [ 8  0  0 ...  0  0  0]
 [ 7  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [36]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [43]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [45]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=10, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)

# evaluate the model
loss, accuracy = model.evauate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 10, 100)           3800      
_________________________________________________________________
flatten_13 (Flatten)         (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 1001      
Total params: 4,801
Trainable params: 1,001
Non-trainable params: 3,800
_________________________________________________________________
None


ValueError: Input arrays should have the same number of samples as target arrays. Found 687377 input samples and 1301 target samples.

In [15]:
disease_df = pd.read_csv('NCBItrainset.csv')
labels = disease_df['diseaseName']

n = 0
label_index = []
for i in labels:
    label_index.append(n)
    n = n+1

label_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [29]:
import fileinput

def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f.read():
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = processLine(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})

In [32]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

# fit the model
predictions = model.evaluate_generator(padded_docs, steps=10000,  max_queue_size=10, 
                                  workers=1, use_multiprocessing=False, verbose=0)
predictions
# evaluate the model
#loss, accuracy = model.evaluate(padded_docs, verbose=0)
#print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 10, 8)             66120     
_________________________________________________________________
flatten_11 (Flatten)         (None, 80)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 81        
Total params: 66,201
Trainable params: 66,201
Non-trainable params: 0
_________________________________________________________________
None


TypeError: 'numpy.ndarray' object is not an iterator

In [2]:
import keras

Using Theano backend.


In [3]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

In [5]:
data = text_to_word_sequence(lines)
print(data)

['wnt', 'signalling', 'orchestrates', 'a', 'number', 'of', 'developmental', 'programs', 'in', 'response', 'to', 'this', 'stimulus', 'cytoplasmic', 'beta', 'catenin', 'encoded', 'by', 'ctnnb1', 'is', 'stabilized', 'enabling', 'downstream', 'transcriptional', 'activation', 'by', 'members', 'of', 'the', 'lef', 'tcf', 'family', 'one', 'of', 'the', 'target', 'genes', 'for', 'beta', 'catenin', 'tcf', 'encodes', 'c', 'myc', 'explaining', 'why', 'constitutive', 'activation', 'of', 'the', 'wnt', 'pathway', 'can', 'lead', 'to', 'cancer', 'particularly', 'in', 'the', 'colon', 'most', 'colon', 'cancers', 'arise', 'from', 'mutations', 'in', 'the', 'gene', 'encoding', 'adenomatous', 'polyposis', 'coli', 'apc', 'a', 'protein', 'required', 'for', 'ubiquitin', 'mediated', 'degradation', 'of', 'beta', 'catenin', 'but', 'a', 'small', 'percentage', 'of', 'colon', 'and', 'some', 'other', 'cancers', 'harbour', 'beta', 'catenin', 'stabilizing', 'mutations', 'recently', 'we', 'discovered', 'that', 'transgenic

In [6]:
# estimate the size of the vocabulary
words = set(text_to_word_sequence(lines))
vocab_size = len(words)
print(vocab_size)

8265


In [38]:
words

{'loxp',
 'xp22',
 'examined',
 'multiprotein',
 'immuno',
 '476',
 'xj1',
 'ferricytochrome',
 'n224d',
 'precocious',
 'fd',
 'advantages',
 'ea',
 'agrees',
 'checkpoints',
 'merzbacher',
 'modifiers',
 'purified',
 'striated',
 'lacks',
 '0099',
 'rationalize',
 'catabolism',
 'clarify',
 'spans',
 'fc',
 'fadd',
 'rectl',
 'last',
 'leucocytes',
 'nitroblue',
 'i65t',
 'innermost',
 'mutants',
 'integrity',
 'shirin',
 'asians',
 'apoptotic',
 'genetically',
 'anosmia',
 'laser',
 'permanent',
 'pose',
 'postoperative',
 '273ser',
 'rgg',
 'combination',
 'oxidation',
 'better',
 'retinitis',
 'secretory',
 'petrich',
 'mix',
 'destroyed',
 'ependymomas',
 'loci',
 'induce',
 'premature',
 'definition',
 'meioses',
 'sibs',
 'ancestor',
 'racial',
 'guarantee',
 'pressure',
 'consecutive',
 'lies',
 'thorough',
 'cause',
 'adults',
 'blot',
 'c2',
 'ata',
 '08',
 'described',
 'factor',
 'rash',
 'thomas',
 'technetium',
 'ala305',
 'documents',
 'origin',
 'symptoms',
 'neurophil

In [7]:
from keras.preprocessing.text import one_hot

# integer encode the document
result = one_hot(lines, round(vocab_size*1.3))
print(result)

[2507, 2572, 4486, 10301, 10316, 4920, 9933, 7165, 7423, 8345, 3287, 10090, 3033, 991, 3932, 3961, 6678, 9573, 3255, 9828, 197, 8237, 3394, 5905, 4694, 9573, 6759, 4920, 9367, 4802, 9687, 5827, 8073, 4920, 9367, 5254, 4120, 1484, 3932, 3961, 9687, 2799, 6668, 3356, 7337, 9079, 113, 4694, 4920, 9367, 2507, 909, 9268, 354, 3287, 10268, 7210, 7423, 9367, 15, 8344, 15, 5957, 9649, 5108, 440, 7423, 9367, 5077, 8693, 8230, 3313, 6932, 7051, 10301, 1914, 2362, 1484, 6320, 9441, 6630, 4920, 3932, 3961, 847, 10301, 6317, 1347, 4920, 15, 2485, 4192, 4138, 5957, 71, 3932, 3961, 5361, 440, 689, 5453, 3333, 6721, 10123, 8847, 7743, 8886, 4922, 3932, 3961, 6053, 1247, 3287, 2517, 1218, 2435, 2081, 8293, 7206, 6721, 9367, 1218, 4920, 9877, 5591, 8847, 4450, 4659, 854, 4920, 4894, 2705, 1724, 3446, 2117, 5453, 2902, 2929, 7583, 8293, 1749, 9229, 5108, 1724, 4077, 6062, 2485, 2929, 491, 1749, 5641, 3932, 3961, 5361, 440, 7824, 5453, 6276, 9367, 2844, 7095, 2485, 7209, 4920, 10090, 4380, 7583, 1218, 678

In [18]:
from keras.preprocessing.text import hashing_trick

# integer encode the document
result1 = hashing_trick(lines, round(vocab_size*1.3), hash_function='md5')
print(result1)

[388, 7500, 1686, 6845, 10224, 2913, 5266, 8998, 662, 4012, 5936, 581, 675, 1809, 2580, 3399, 4894, 9892, 9530, 6486, 3966, 2423, 8505, 4136, 9348, 9892, 1937, 2913, 7113, 9710, 3831, 2355, 5649, 2913, 7113, 5513, 1540, 6655, 2580, 3399, 3831, 2167, 2357, 3338, 9246, 5599, 1176, 9348, 2913, 7113, 388, 9708, 1611, 6247, 5936, 7145, 7208, 662, 7113, 1465, 3290, 1465, 10728, 90, 9946, 9952, 662, 7113, 5923, 930, 6457, 7599, 6442, 8406, 6845, 733, 2761, 6655, 6729, 6293, 29, 2913, 2580, 3399, 2018, 6845, 7082, 7293, 2913, 1465, 2670, 6994, 5660, 10728, 9219, 2580, 3399, 2718, 9952, 6546, 5265, 1860, 8868, 10146, 8877, 3701, 10185, 2743, 2580, 3399, 2714, 4280, 5936, 8127, 6723, 8908, 7960, 1219, 5937, 8868, 7113, 6723, 2913, 2294, 3269, 8877, 7211, 10401, 4744, 2913, 3089, 5538, 5727, 2575, 2695, 5265, 8714, 927, 5455, 1219, 2007, 2547, 9946, 5727, 5859, 4258, 2670, 927, 4652, 2007, 8746, 2580, 3399, 2718, 9952, 10659, 5265, 7999, 7113, 2089, 7046, 2670, 8252, 2913, 581, 7327, 5455, 6723, 

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

max_words = 10000

model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.metrics_names)

['loss', 'acc']


In [33]:
batch_size = 10000
epochs = 3000

history = model.fit(encod_docs, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
score = model.evaluate(encod_docs, batch_size=batch_size, verbose=1)
#output = model.predict(encoded_docs, batch)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
#output

ValueError: Error when checking input: expected dense_3_input to have shape (10000,) but got array with shape (38,)

In [9]:
#create dictionary using annotations
 
import csv
import sys
import pprint

with open('NCBItrainset.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('dict.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        mydict = {rows[0]:rows[1] for rows in reader}

In [51]:
disease_df = pd.read_csv('NCBItrainset.csv')

In [45]:
y = ['DiseaseClass', 'SpecificDisease', 'CompositeMention', 'Modifier']

In [52]:
labels = disease_df['diseaseName']

In [53]:
labels

0                                             skin tumour
1                                                  cancer
2                                           colon cancers
3                              adenomatous polyposis coli
4                                                     APC
5                            colon and some other cancers
6                                            skin tumours
7                                          pilomatricomas
8                                          pilomatricomas
9                                             skin tumour
10                                                 tumour
11                                         pilomatricomas
12                                                tumours
13                                                tumours
14                                        hemochromatosis
15                                        hemochromatosis
16                             Hereditary hemochromatosis
17            

In [1]:
Y = []
for i in y:
    if i == 'DiseaseClass':
        Y.append(0)
    elif i == 'SpecificDisease':
        Y.append(1)
    elif i == 'Modifier':
        Y.append(2)
    else:
        Y.append(3)

NameError: name 'y' is not defined

In [ ]:
y

In [38]:
mydict

{'A-T': 'SpecificDisease',
 'AAPC': 'Modifier',
 'AGU': 'Modifier ',
 'AKU': 'Modifier ',
 'ALD': 'SpecificDisease',
 'ALPS': 'Modifier ',
 'AMN': 'Modifier ',
 'APC': 'Modifier ',
 'Abnormalities in the HDL': 'DiseaseClass',
 'Absence of the seventh component of complement': 'SpecificDisease',
 'Addison disease': 'SpecificDisease',
 'Addison only': 'SpecificDisease',
 'Addisons disease': 'SpecificDisease',
 'Adrenal insufficiency': 'DiseaseClass',
 'Adrenoleukodystrophy': 'SpecificDisease',
 'Alkaptonuria': 'SpecificDisease',
 'Aspartylglucosaminuria': 'SpecificDisease',
 'Autoimmune lymphoproliferative syndrome': 'SpecificDisease',
 'Autosomal recessive familial neurohypophyseal diabetes insipidus': 'SpecificDisease',
 'BMD': 'Modifier',
 'BPAD': 'SpecificDisease ',
 'Brugada syndrome': 'SpecificDisease',
 'C2-deficient': 'Modifier',
 'C3 deficiency': 'SpecificDisease ',
 'C5 deficiency': 'SpecificDisease',
 'C6 deficiency': 'SpecificDisease ',
 'C7 deficiency': 'SpecificDisease',
 '

In [39]:
from keras_text.processing import WordTokenizer

tokenizer = WordTokenizer()
tokenizer.build_vocab(lines)

687377/687377 [==============================] - ETA:  - ETA: 31:2 - ETA: 17:3 - ETA: 13:0 - ETA: 10:5 - ETA: 9:2 - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA:

In [40]:
from keras_text.models import TokenModelFactory
from keras_text.models import YoonKimCNN, AttentionRNN, StackedRNN


# RNN models can use `max_tokens=None` to indicate variable length words per mini-batch.
factory = TokenModelFactory(1, tokenizer.token_index, max_tokens=100, embedding_type='glove.6B.100d')
word_encoder_model = YoonKimCNN()
model = factory.build_model(token_encoder_model=word_encoder_model)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

ImportError: No module named 'token_model'

# preprocessing

In [48]:
"""
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

NameError: name 'os' is not defined

In [45]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.utils 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences)#, maxlen=MAX_SEQUENCE_LENGTH)


Found 37 unique tokens.


In [63]:
from keras.utils import to_categorical
n = 0
label_index = []
for i in labels:
    label_index.append(n)
    n = n+1

label_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [64]:
label = to_categorical(np.asarray(label_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (687377, 1)
Shape of label tensor: (1301,)
